In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow.keras 
from tensorflow.keras import models
from skimage.transform import resize
import matplotlib.image as image
from skimage import color

In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    dcm = pydicom.dcmread(filename)       
   
    # Age
    age = int(dcm.PatientAge)
    if age < 2 or age > 90:
        print(f'Image {filename} rejected, Age: {age}')
        return None
    
    # Body Part
    body_part = dcm.BodyPartExamined
    if body_part not in ['CHEST', 'chest']:
        print(f'Image {filename} rejected, Body part: {body_part}')
        return None
    
    # Patient Position
    pos = dcm.PatientPosition
    if pos not in ['PA', 'AP']:
        print(f'Image {filename} rejected, Patient position: {pos}')
        return None
    
    # Modali
    mod = dcm.Modality                            
    if mod != 'DX':
        print(f'Image {filename} rejected, Modality: {mod}')
        return None
    
    return dcm.pixel_array
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_size):
    if len(img.shape) == 3:
        img = color.rgb2gray(img)
    # Normalize image
    img  = img / 255
    resized_img = resize(img, (img_size[1], img_size[2]))

    proc_img = resized_img.reshape((1, img_size[1], img_size[2], 1))
    proc_img = np.repeat(proc_img, img_size[3], axis=3)

    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    with open(model_path, 'r') as json_file:
        model_json = json_file.read()
        model = models.model_from_json(model_json)
        model.load_weights(weight_path)    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    pred_im = model.predict(img)   
    if pred_im > thresh:
        prediction = 'Pneumonia'
    else:
        prediction =  'Not Pneumonia'    
    return prediction 

In [3]:
test_dicoms = ['./data/test1.dcm','./data/test2.dcm','./data/test3.dcm','./data/test4.dcm','./data/test5.dcm','./data/test6.dcm']

model_path = './my_model.json' #path to saved model
weight_path = './xray_class_my_model.best.hdf5' #path to saved best weights

IMG_SIZE=(1,224,224,3) 

In [4]:
my_model = load_model(model_path, weight_path) #loads model
thresh = 0.36

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Not Pneumonia
Pneumonia
Pneumonia
Image ./data/test4.dcm rejected, Body part: RIBCAGE
Image ./data/test5.dcm rejected, Modality: CT
Image ./data/test6.dcm rejected, Patient position: XX
